In [ ]:
#!pip install tensorflow==2.5.0

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [ ]:
def w1():
  '''
  1 layer 1 뉴런, input_shape가 한가지 value만 가지고 있을때
  가장 기본적으로는 아래와 같이 정의하여 사용한다.
  model.save('mnist_mlp_model.h5') 이렇게 쓰면 h5 저장 가능
  '''

  model = tf.keras.Sequential([keras.layers.Dense(units=1, input_shape=[1])])
  model.compile(optimizer='sgd', loss='mean_squared_error')

In [2]:
# GRADED FUNCTION: house_model
def house_model(y_new):
    xs = np.array([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], dtype=float)
    ys = np.array([1.0, 1.5, 2.0, 2.5, 3.0, 3.5], dtype=float)

    # alternate solution
    #xs = np.array([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], dtype=float)
    #ys = 0.5 * (xs + 1)
    
    model = tf.keras.Sequential([keras.layers.Dense(units=1, input_shape=[1])])
    model.compile(optimizer='sgd', loss='mean_squared_error')
    model.fit(xs, ys, epochs=500)
    
    return model.predict(y_new)[0]
prediction = house_model([7.0])
print(prediction)

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') is not None and logs.get('accuracy') >= 0.8): # Experiment with changing this value
      print("\nReached 80% accuracy so cancelling training!")
      self.model.stop_training = True

In [ ]:


def w2():
  '''
  mnist로 데이터 불러옴
  '''

  callbacks = myCallback()

  mnist = tf.keras.datasets.fashion_mnist
  (training_images, training_labels), (test_images, test_labels) = mnist.load_data()

  print(training_images.shape) #이걸로 이미지 사이즈 확인할 수 있음

  # print(training_labels[0]) #label 10개
  # print(training_images[0])


  # 데이터를 0~1로 셋팅해준다
  training_images  = training_images / 255.0
  test_images = test_images / 255.0

  # flat으로 데이터를 한줄로 쭉 펴준다. 이미지를 28,28 into a 784x1
  # input_shape=(28, 28)을 쓸때도 있고 안쓸때도 있고... 인듯

  # relu는 0보다 클때 그 값 리턴 : 앞에 숫자를 크게 하면, 뉴런을 더 추가하는 거라서 더 많은 연산을 한다
  # 항상 적절하게 테스트해야지 너무 많이 하면 과적합 상태가 됨

  # softmax는 주어진 범위에서 가장 큰 값 : 가장 확률 높은애, 따라서 앞에 10은 label 수로 들어가게 된다.

  # sigmoid는 yes or no처럼 이진일때 효과 좋음

  # 
  model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)), 
    tf.keras.layers.Dense(128, activation=tf.nn.relu), 
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    
  ])
  
  model.summary()
  
  #model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy') 이것도 된다
  model.compile(optimizer = tf.optimizers.Adam(),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

  # model.fit(training_images, training_labels, epochs=5)

  #콜백 사용
  model.fit(training_images, training_labels, epochs=5, callbacks=[callbacks])

  #요거로 테스트 진행
  model.evaluate(test_images, test_labels)
w2()


In [ ]:
def w3():
  mnist = tf.keras.datasets.fashion_mnist
  (training_images, training_labels), (test_images, test_labels) = mnist.load_data()

  callbacks = myCallback()

  # 60000장 28x28 이미지를 변환한다.
  training_images=training_images.reshape(60000, 28, 28, 1)
  training_images=training_images / 255.0

  test_images = test_images.reshape(10000, 28, 28, 1)
  test_images=test_images/255.0

  # 보통 3x3이나 5x5 배열로 주위 애들이랑 연산한다
  # Conv2d 파라미터로 젓번째는 생성하고자 하는 convolution 수다. 보통 32, 64 사용 kernel 필터의 개수
  # 필터가 많을수록 더 많은 경우의 수를 본다
  # 3,3은 생성하고자 하는 배열

  # Maxpooling2D는 크기를 줄여나가는 것이다. 2x2 중에서 가장 큰 1개만 추출한다
  # 따라서 사이즈가 25프로 줄어듬 (가로 반, 세로 반)

  # 사이즈를 줄이고, 마지막에 Flatten을 실행시켜줘야 

  model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
  ])
  model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  model.summary()
  history = model.fit(
      # YOUR CODE STARTS HERE
      training_images,
      training_labels,
      epochs = 10,
      callbacks = [callbacks]
      # YOUR CODE ENDS HERE
  )
  test_loss = model.evaluate(test_images, test_labels)

  return history.epoch, history.history['acc'][-1]

w3()

In [ ]:

def w4():
    DESIRED_ACC = 0.999
    class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if(logs.get('acc')>DESIRED_ACC):
                print("\n\nI did it!!")
                self.model.stop_training = True

    callbacks = myCallback()
    
    # This Code Block should Define and Compile the Model. Please assume the images are 150 X 150 in your implementation.
    model = tf.keras.models.Sequential([
        # convolutional layer
        tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
        tf.keras.layers.MaxPooling2D(2,2),

        tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),

        tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),

        # DNN 
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid') #binary classification
        # Your Code Here
    ])

    from tensorflow.keras.optimizers import RMSprop

    # model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=0.001), metrics=['accuracy'])
    model.compile(optimizer='RMSprop', loss='binary_crossentropy', metrics=['accuracy'])
        

    # This code block should create an instance of an ImageDataGenerator called train_datagen 
    # And a train_generator by calling train_datagen.flow_from_directory

    from tensorflow.keras.preprocessing.image import ImageDataGenerator

    train_datagen = ImageDataGenerator(rescale=1./255)

    # Please use a target_size of 150 X 150.
    train_generator = train_datagen.flow_from_directory(
        '/tmp/h-or-s',
        target_size=(150,150),
        batch_size=8,
        class_mode='binary')  # binary_crossentropy loss를 사용하므로 binary labels 필요
    # Expected output: 'Found 80 images belonging to 2 classes'

    history = model.fit_generator(
        train_generator,
        steps_per_epoch=10,
        epochs=20, # 20번 돌리고 0.999 이상 나오면 멈추도록
        verbose=1,
        callbacks=[callbacks])
    # model fitting
    return history.history['acc'][-1]